In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/elastic_df.csv")
name_df = df[["name", "tags"]]
name_df["name_splitted"] = name_df['name'].apply(lambda x: x.split(" "))

<ipython-input-2-0012758c228c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df["name_splitted"] = name_df['name'].apply(lambda x: x.split(" "))


In [3]:
name_df.head()

,name,tags,name_splitted
0,High Court,"['opening_hours|10:00-17:00', 'office|governme...","[High, Court]"
1,District Technical Assistance Office,"['opening_hours|10:00-17:00', 'office|governme...","[District, Technical, Assistance, Office]"
2,Peace Zone Academy,"['opening_hours|Su-Th 10:00-16:00, Fr 10:00-13...","[Peace, Zone, Academy]"
3,Holi Angel School,"['phone|9846001898', 'opening_hours|Su-Th 10:0...","[Holi, Angel, School]"
4,Phewa Public Secondary Higher School,"['phone|061-536144', 'opening_hours|10:00-17:0...","[Phewa, Public, Secondary, Higher, School]"


In [4]:
bow = " ".join(name_df['name'].values)

In [5]:
bow = bow.lower()
bow = bow.split(" ")

In [6]:
from collections import Counter
counter = Counter(bow)
print(counter)

Counter({'school': 4766, 'shree': 4731, 'bank': 2465, 'and': 2420, 'hotel': 2371, 'pra': 2244, 'v': 2080, 'secondary': 1645, 'restaurant': 1617, 'nepal': 1612, 'limited': 1585, 'vi': 1581, '&': 1462, 'house': 1368, 'ma': 1355, 'mandir': 1339, 'office': 1065, 'khola': 1058, 'lodge': 1047, 'temple': 1040, 'ward': 988, 'primary': 874, 'guest': 733, 'center': 696, 'of': 689, 'english': 677, 'cafe': 635, 'higher': 611, 'chowk': 601, 'academy': 585, 'devi': 582, 'hospital': 576, 'ni': 568, 'boarding': 556, 'ghar': 554, 'home': 538, 'new': 533, 'ltd': 513, 'health': 506, 'shop': 489, 'view': 482, 'municipality': 471, 'medical': 465, 'store': 460, 'college': 455, 'centre': 449, 'gaunpalika': 443, 'post': 437, 'ganesh': 424, 'international': 420, 'pvt.': 420, 'police': 419, 'ltd.': 412, 'development': 405, 'the': 400, 'pasal': 389, 'khaja': 384, 'chautari': 382, 'pharmacy': 382, 'shiva': 376, 'atm': 372, 'credit': 367, 'jyoti': 366, 'farm': 356, 'kalika': 351, 'river': 345, 'himalayan': 340, 'h

In [7]:
count_df = pd.DataFrame(counter.items(), columns=["text", "count"]).sort_values(by='count', ascending=False)

In [8]:
count_df["status"] = 0
count_df["incorrect"] = 0
count_df['g_translated'] = 0
count_df['google_translate'] = ''

In [9]:
count_df.head()

,text,count,status,incorrect,g_translated,google_translate
11,school,4766,0,0,0,
29,shree,4731,0,0,0,
39,bank,2465,0,0,0,
68,and,2420,0,0,0,
69,hotel,2371,0,0,0,


In [10]:
# !pip install requests
# !pip install swifter
# import swifter
import requests, time
from urllib3.connection import SocketTimeout
def translate_to_np(text):
    state = count_df[count_df['text']==text]
    index = state.index.values[0]
    if state['g_translated'].values[0]==1:
        return count_df.loc[index]["google_translate"]
    try:
        translate_api_url = f"https://translate.googleapis.com/translate_a/single?client=gtx&sl=en&tl=ne&dt=t&q={text}"
        response = requests.get(translate_api_url)
        # print(response.status_code, response.json()[0][0])
    except SocketTimeout:
        return "retry"
    if response.status_code==200:
        try:
            ne_text = response.json()[0][0][0]
            count_df.loc[count_df["text"]==text, "g_translated"]=1
        except (TypeError, KeyError):
            return "none"
    else:
        ne_text = "error"
    # time.sleep(5)
    print(ne_text)
    return ne_text

    # Commenting this step as this takes a lot of time to complete. Output of this step has been saved in csv and will be loaded in df in next step
    # count_df["google_translate"] = count_df["text"].swifter.apply(lambda x: translate_to_np(x))

In [18]:
translated_df = pd.read_csv("/content/drive/MyDrive/translated.csv")

In [19]:
translated_df.head()

,text,count,status,incorrect,g_translated,google_translate
0,school,4766,0,0,1,विद्यालय
1,shree,4731,0,0,1,श्री
2,bank,2465,0,0,1,बैंक
3,and,2420,0,0,1,र
4,hotel,2371,0,0,1,होटल


In [38]:
translated_df[translated_df['g_translated']==0]

,text,count,status,incorrect,g_translated,google_translate,corrected,text_ne
12,&,1462,0,0,0,none,,
301,NaN,77,0,0,0,none,,
1300,#,15,0,0,0,none,,
4318,+,4,0,0,0,none,,
6986,#1,2,0,0,0,none,,
8037,&female,2,0,0,0,none,,
10840,++,1,0,0,0,none,,
12608,#3,1,0,0,0,none,,
13677,#bor,1,0,0,0,none,,
14304,&homestay,1,0,0,0,none,,


In [21]:
translated_df["corrected"] = ""

In [22]:
translated_df["text_ne"] = ""

In [36]:
translated_df.head()

,text,count,status,incorrect,g_translated,google_translate,corrected,text_ne
0,school,4766,0,0,1,विद्यालय,,
1,shree,4731,0,0,1,श्री,,
2,bank,2465,0,0,1,बैंक,,
3,and,2420,0,0,1,र,,
4,hotel,2371,0,0,1,होटल,,


In [39]:
translated_df.to_csv("name_df.csv", index=False)